In [271]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [272]:
data = pd.read_csv("isolates.csv")
data.head(10)

,#Organism group,Isolate,AMR genotypes,AST phenotypes
0,Listeria monocytogenes,PDT000077416.3,"fosX=COMPLETE,lin=COMPLETE","chloramphenicol=S,clindamycin=R,erythromycin=S..."
1,Listeria monocytogenes,PDT000095192.3,"fosX=COMPLETE,lin=COMPLETE","ampicillin=S,penicillin=S"
2,Salmonella enterica,PDT000003687.3,"mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
3,Salmonella enterica,PDT000003688.4,"mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
4,Salmonella enterica,PDT000003689.4,"mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
5,Salmonella enterica,PDT000003690.3,"aph(3'')-Ib=COMPLETE,aph(6)-Id=COMPLETE,mdsA=C...","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
6,Salmonella enterica,PDT000003691.3,"mdsA=COMPLETE,mdsB=COMPLETE,tet(B)=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
7,Salmonella enterica,PDT000003692.3,"mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
8,Salmonella enterica,PDT000003693.3,"aph(3'')-Ib=COMPLETE,aph(6)-Id=COMPLETE,mdsA=C...","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."
9,Salmonella enterica,PDT000003694.4,"fosA7=COMPLETE,mdsA=COMPLETE,mdsB=COMPLETE","amikacin=S,amoxicillin-clavulanic acid=S,ampic..."


In [273]:
def transform_dataframe(df):
    new_data = []

    for idx, row in df.iterrows():
        # split AMR genotypes and remove "=COMPLETE"
        amr_genotypes = [i.split('=')[0] for i in row['AMR genotypes'].split(',')]

        # split AST phenotypes
        ast_phenotypes = row['AST phenotypes'].split(',')

        # process each phenotype
        for pheno in ast_phenotypes:
            drug, resistance = pheno.split('=')
            if resistance == 'R':
                resistance_score = 1
            elif resistance == 'S':
                resistance_score = 0
            else:  # assuming 'I' as per your description
                resistance_score = 0.5

            # create a new row
            new_row = {'#Organism group': row['#Organism group'],
                       'Isolate': row['Isolate'],
                       'AMR genotypes': ', '.join(amr_genotypes),
                       'drug': drug,
                       'resistance': resistance_score}
            new_data.append(new_row)

    # create a new dataframe
    new_df = pd.DataFrame(new_data)

    return new_df


In [274]:
df = transform_dataframe(data)


In [275]:
df.head(10)

,#Organism group,Isolate,AMR genotypes,drug,resistance
0,Listeria monocytogenes,PDT000077416.3,"fosX, lin",chloramphenicol,0.0
1,Listeria monocytogenes,PDT000077416.3,"fosX, lin",clindamycin,1.0
2,Listeria monocytogenes,PDT000077416.3,"fosX, lin",erythromycin,0.0
3,Listeria monocytogenes,PDT000077416.3,"fosX, lin",gentamicin,0.0
4,Listeria monocytogenes,PDT000077416.3,"fosX, lin",levofloxacin,0.0
5,Listeria monocytogenes,PDT000077416.3,"fosX, lin",oxacillin,1.0
6,Listeria monocytogenes,PDT000077416.3,"fosX, lin",penicillin,0.0
7,Listeria monocytogenes,PDT000077416.3,"fosX, lin",rifampin,0.0
8,Listeria monocytogenes,PDT000077416.3,"fosX, lin",tetracycline,0.0
9,Listeria monocytogenes,PDT000077416.3,"fosX, lin",trimethoprim-sulfamethoxazole,0.0


In [276]:
df.shape

(316071, 5)

In [277]:
df.to_csv("BasicData.csv",index=False)

In [278]:
unique_all_genes = set()
unique_all_antibiotics = set()

def create_empty_gene_antibiotic_df(df):
    for idx, row in df.iterrows():
        genotypes = row['AMR genotypes'].split(', ')
        antibiotic = row['drug']
        
        unique_all_genes.update(genotypes)
        unique_all_antibiotics.add(antibiotic)
    
    gene_antibiotic_df = pd.DataFrame(index=unique_all_genes, columns=unique_all_antibiotics)
    gene_antibiotic_df = gene_antibiotic_df.fillna(-1)
    
    return gene_antibiotic_df

# Assuming your DataFrame is named 'new_dataframe'
gene_antibiotic_df = create_empty_gene_antibiotic_df(new_dataframe)
gene_antibiotic_df



,chloramphenicol,dicloxacillin,ciprofloxacin,ceftiofur,fosfomycin-glucose-6-phosphate,amoxicillin-clavulanic acid,benzylpenicillin,metronidazole,linezolid,piperacillin,...,trimethoprim-sulfamethoxazole,aztreonam,norfloxacin,neomycin,Imipenem-EDTA-PA,delafloxacin,zoliflodacin,vancomycin,ertapenem,cefiderocol
qnrB48,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
blaADC-155,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
oqxB19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
blaACT-37,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
blaTEM-19,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
blaOXA-494,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
qnrB38,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
tmexD3,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
tet(O),-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


***********************************************

In [279]:
def gene_per_drug(drug):
    drug_df = df[df['drug'] == drug]
    unique_genes = set()
    for genes in drug_df['AMR genotypes'].str.split(', '):
        unique_genes.update(genes)
    return drug_df, unique_genes

In [280]:
def create_gene_df(drug_df, unique_genes):
    gene_arrays = []
    labels = []

    for idx, row in drug_df.iterrows():
        gene_array = np.zeros(len(unique_genes), dtype=int)
        genes = row['AMR genotypes'].split(', ')
        for gene in genes:
            gene_index = list(unique_genes).index(gene)
            gene_array[gene_index] = 1

        gene_arrays.append(gene_array.tolist())
        labels.append(row['resistance'])

    gene_df = pd.DataFrame(gene_arrays, columns=unique_genes)
    gene_df['label'] = labels

    return gene_df


In [281]:

def Model(drug_df_for_model, epochs, batch_size):
    # Assuming your DataFrame with gene features and labels is named `oxacillin_df_for_model`
    # Split the data into features (genes) and labels
    features = drug_df_for_model.drop('label', axis=1).values
    labels = drug_df_for_model['label'].values

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Define the neural network model
    model = keras.Sequential()
    model.add(keras.layers.Dense(64, activation='relu', input_shape=(features.shape[1],)))
    model.add(keras.layers.Dense(32, activation='relu'))
    model.add(keras.layers.Dense(16, activation='relu'))
    model.add(keras.layers.Dense(8, activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))  # Sigmoid activation for probability between 0 and 1

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mse'])

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

    # Evaluate the model
    loss, mse = model.evaluate(X_test, y_test)
    pred = model.predict(X_test)
    print('Loss:', loss)
    print('Mean Squared Error:', mse)
    print('len of pred: ', pred.shape, 'y_test: ', y_test.shape)
    # Make predictions
    return y_test, pred, model

In [282]:
def evaluation(y_test, predictions, drug):
    # Calculate mean squared error
    mse = mean_squared_error(y_test, predictions)

    # Calculate mean absolute error
    mae = mean_absolute_error(y_test, predictions)

    # Calculate R^2 score
    r2 = r2_score(y_test, predictions)

    
    print('Mean Squared Error:', mse)
    print('Mean Absolute Error:', mae)
    print('R^2 Score:', r2)
    return r2


In [283]:
def Create_df_one_gene(drug_df_for_model):
    # Extract the column names (excluding the label column)
    column_names = drug_df_for_model.columns[:-1]

    # Create a new DataFrame with zeros
    df_one_gene = pd.DataFrame(0, index=np.arange(len(column_names)), columns=column_names)

    # Set the value at the corresponding index position in each row
    for i in range(len(column_names)):
        df_one_gene.iloc[i, i] = 1

    # Display the new DataFrame
    return df_one_gene

In [284]:
def PredR_Antibiotic(drug):
    print('************************\n', drug)
    
    drug_df,drug_genes =   gene_per_drug(drug)
    drug_df_for_model = create_gene_df(drug_df, drug_genes)
    y_test, predictions, model = Model(drug_df_for_model, 20, 10)
    r2 = evaluation(y_test, predictions, drug)
    df_gene = Create_df_one_gene(drug_df_for_model)
    pred = model.predict(df_gene)
    for i, col in enumerate(df_gene.columns):
        gene_antibiotic_df.loc[col, drug] = pred[i]
    return r2

In [285]:
antibiotic_few_samples = []
antibiotic_precent = {} 
for antibiotic in unique_all_antibiotics:
    if len(df[df['drug'] == antibiotic]) < 50:
        antibiotic_few_samples.append(antibiotic)
        continue   
    r2 = PredR_Antibiotic(antibiotic)
    antibiotic_precent[antibiotic] = r2

************************
 chloramphenicol
Epoch 1/20
982/982 [==============================] - 1s 956us/step - loss: 0.1173 - mse: 0.0267
Epoch 2/20
982/982 [==============================] - 1s 978us/step - loss: 0.0525 - mse: 0.0065 0s - loss: 0.0550 - mse: 
Epoch 3/20
982/982 [==============================] - 1s 989us/step - loss: 0.0466 - mse: 0.0051
Epoch 4/20
982/982 [==============================] - 1s 996us/step - loss: 0.0449 - mse: 0.0048
Epoch 5/20
982/982 [==============================] - 1s 1ms/step - loss: 0.0423 - mse: 0.0044
Epoch 6/20
982/982 [==============================] - 1s 1ms/step - loss: 0.0437 - mse: 0.0049
Epoch 7/20
982/982 [==============================] - 1s 1ms/step - loss: 0.0408 - mse: 0.0041
Epoch 8/20
982/982 [==============================] - 1s 987us/step - loss: 0.0401 - mse: 0.0041
Epoch 9/20
982/982 [==============================] - 1s 1ms/step - loss: 0.0404 - mse: 0.0043
Epoch 10/20
982/982 [==============================] - 1s 1ms/step 

31/31 [==============================] - 0s 617us/step - loss: 0.0913 - mse: 0.0105
Loss: 0.0913119688630104
Mean Squared Error: 0.010456227697432041
len of pred:  (971, 1) y_test:  (971,)
Mean Squared Error: 0.010456228171436235
Mean Absolute Error: 0.016321102735249643
R^2 Score: 0.9201845849123188
************************
 amoxicillin-clavulanic acid
Epoch 1/20
1173/1173 [==============================] - 1s 894us/step - loss: 0.1910 - mse: 0.0372
Epoch 2/20
1173/1173 [==============================] - 1s 876us/step - loss: 0.1254 - mse: 0.0174
Epoch 3/20
1173/1173 [==============================] - 1s 891us/step - loss: 0.1169 - mse: 0.0150
Epoch 4/20
1173/1173 [==============================] - 1s 890us/step - loss: 0.1130 - mse: 0.0139
Epoch 5/20
1173/1173 [==============================] - 1s 898us/step - loss: 0.1101 - mse: 0.0133
Epoch 6/20
1173/1173 [==============================] - 1s 898us/step - loss: 0.1073 - mse: 0.0125
Epoch 7/20
1173/1173 [============================

72/72 [==============================] - 0s 903us/step - loss: 0.4421 - mse: 0.0045
Epoch 9/20
72/72 [==============================] - 0s 920us/step - loss: 0.4362 - mse: 0.0030
Epoch 10/20
72/72 [==============================] - 0s 840us/step - loss: 0.4345 - mse: 0.0024
Epoch 11/20
72/72 [==============================] - 0s 878us/step - loss: 0.4332 - mse: 0.0023
Epoch 12/20
72/72 [==============================] - 0s 881us/step - loss: 0.4313 - mse: 0.0019
Epoch 13/20
72/72 [==============================] - 0s 894us/step - loss: 0.4309 - mse: 0.0017
Epoch 14/20
72/72 [==============================] - 0s 902us/step - loss: 0.4291 - mse: 0.0013
Epoch 15/20
72/72 [==============================] - 0s 906us/step - loss: 0.4279 - mse: 0.0010
Epoch 16/20
72/72 [==============================] - 0s 903us/step - loss: 0.4272 - mse: 8.0878e-04
Epoch 17/20
72/72 [==============================] - 0s 879us/step - loss: 0.4270 - mse: 8.1433e-04
Epoch 18/20
72/72 [==========================

293/293 [==============================] - 0s 895us/step - loss: 0.0543 - mse: 0.0099
Epoch 17/20
293/293 [==============================] - 0s 889us/step - loss: 0.0539 - mse: 0.0101
Epoch 18/20
293/293 [==============================] - 0s 883us/step - loss: 0.0531 - mse: 0.0099
Epoch 19/20
293/293 [==============================] - 0s 902us/step - loss: 0.0534 - mse: 0.0099
Epoch 20/20
23/23 [==============================] - 0s 786us/step - loss: 0.1856 - mse: 0.0312
Loss: 0.18558287620544434
Mean Squared Error: 0.03122231923043728
len of pred:  (731, 1) y_test:  (731,)
Mean Squared Error: 0.03122232015659518
Mean Absolute Error: 0.04209062051908887
R^2 Score: 0.6034749385937574
************************
 azithromycin
Epoch 1/20
988/988 [==============================] - 1s 931us/step - loss: 0.0719 - mse: 0.0160
Epoch 2/20
988/988 [==============================] - 1s 891us/step - loss: 0.0236 - mse: 0.0041
Epoch 3/20
988/988 [==============================] - 1s 912us/step - loss:

72/72 [==============================] - 0s 866us/step - loss: 0.4946 - mse: 0.0220
Epoch 4/20
72/72 [==============================] - 0s 861us/step - loss: 0.4708 - mse: 0.0140
Epoch 5/20
72/72 [==============================] - 0s 825us/step - loss: 0.4547 - mse: 0.0095
Epoch 6/20
72/72 [==============================] - 0s 868us/step - loss: 0.4452 - mse: 0.0070
Epoch 7/20
72/72 [==============================] - 0s 873us/step - loss: 0.4397 - mse: 0.0060
Epoch 8/20
72/72 [==============================] - 0s 888us/step - loss: 0.4354 - mse: 0.0050
Epoch 9/20
72/72 [==============================] - 0s 903us/step - loss: 0.4319 - mse: 0.0041
Epoch 10/20
72/72 [==============================] - 0s 854us/step - loss: 0.4290 - mse: 0.0033
Epoch 11/20
72/72 [==============================] - 0s 909us/step - loss: 0.4274 - mse: 0.0031
Epoch 12/20
72/72 [==============================] - 0s 878us/step - loss: 0.4271 - mse: 0.0032
Epoch 13/20
72/72 [==============================] - 0s 88

...

62/62 [==============================] - 0s 1ms/step - loss: 0.2109 - mse: 0.0151
Epoch 11/20
62/62 [==============================] - 0s 1ms/step - loss: 0.2068 - mse: 0.0135
Epoch 12/20
62/62 [==============================] - 0s 1ms/step - loss: 0.2117 - mse: 0.0159
Epoch 13/20
62/62 [==============================] - 0s 1ms/step - loss: 0.2034 - mse: 0.0127
Epoch 14/20
62/62 [==============================] - 0s 1ms/step - loss: 0.2031 - mse: 0.0133
Epoch 15/20
62/62 [==============================] - 0s 1ms/step - loss: 0.1999 - mse: 0.0116
Epoch 16/20
62/62 [==============================] - 0s 1ms/step - loss: 0.2039 - mse: 0.0134
Epoch 17/20
62/62 [==============================] - 0s 1ms/step - loss: 0.2027 - mse: 0.0129
Epoch 18/20
62/62 [==============================] - 0s 1ms/step - loss: 0.2000 - mse: 0.0121
Epoch 19/20
62/62 [==============================] - 0s 1ms/step - loss: 0.1933 - mse: 0.0103
Epoch 20/20
5/5 [==============================] - 0s 1ms/step - loss: 0

32/32 [==============================] - 0s 856us/step - loss: 0.4878 - mse: 0.1018
Epoch 3/20
32/32 [==============================] - 0s 878us/step - loss: 0.3872 - mse: 0.0690
Epoch 4/20
32/32 [==============================] - 0s 939us/step - loss: 0.3387 - mse: 0.0510
Epoch 5/20
32/32 [==============================] - 0s 947us/step - loss: 0.3111 - mse: 0.0415
Epoch 6/20
32/32 [==============================] - 0s 971us/step - loss: 0.2864 - mse: 0.0326
Epoch 7/20
32/32 [==============================] - 0s 961us/step - loss: 0.2744 - mse: 0.0288
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 0.2646 - mse: 0.0269
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 0.2533 - mse: 0.0230
Epoch 10/20
32/32 [==============================] - 0s 961us/step - loss: 0.2453 - mse: 0.0211
Epoch 11/20
32/32 [==============================] - 0s 969us/step - loss: 0.2441 - mse: 0.0215
Epoch 12/20
32/32 [==============================] - 0s 959us/s

************************
 synercid
Epoch 1/20
14/14 [==============================] - 0s 862us/step - loss: 0.6376 - mse: 0.2223
Epoch 2/20
14/14 [==============================] - 0s 775us/step - loss: 0.5215 - mse: 0.1657
Epoch 3/20
14/14 [==============================] - 0s 856us/step - loss: 0.3375 - mse: 0.0849
Epoch 4/20
14/14 [==============================] - 0s 871us/step - loss: 0.1487 - mse: 0.0228
Epoch 5/20
14/14 [==============================] - 0s 826us/step - loss: 0.0488 - mse: 0.0035
Epoch 6/20
14/14 [==============================] - 0s 855us/step - loss: 0.0161 - mse: 4.2897e-04
Epoch 7/20
14/14 [==============================] - 0s 795us/step - loss: 0.0073 - mse: 9.0145e-05
Epoch 8/20
14/14 [==============================] - 0s 867us/step - loss: 0.0043 - mse: 3.1727e-05
Epoch 9/20
14/14 [==============================] - 0s 963us/step - loss: 0.0030 - mse: 1.5295e-05
Epoch 10/20
14/14 [==============================] - 0s 904us/step - loss: 0.0022 - mse: 8.669

************************
 norfloxacin
Epoch 1/20
9/9 [==============================] - 0s 937us/step - loss: 0.6936 - mse: 0.2222
Epoch 2/20
9/9 [==============================] - 0s 877us/step - loss: 0.6804 - mse: 0.2156
Epoch 3/20
9/9 [==============================] - 0s 887us/step - loss: 0.6519 - mse: 0.2013
Epoch 4/20
9/9 [==============================] - 0s 882us/step - loss: 0.5858 - mse: 0.1689
Epoch 5/20
9/9 [==============================] - 0s 894us/step - loss: 0.5051 - mse: 0.1314
Epoch 6/20
9/9 [==============================] - 0s 768us/step - loss: 0.4238 - mse: 0.0986
Epoch 7/20
9/9 [==============================] - 0s 884us/step - loss: 0.3596 - mse: 0.0775
Epoch 8/20
9/9 [==============================] - 0s 920us/step - loss: 0.3157 - mse: 0.0634
Epoch 9/20
9/9 [==============================] - 0s 915us/step - loss: 0.2940 - mse: 0.0563
Epoch 10/20
9/9 [==============================] - 0s 776us/step - loss: 0.2721 - mse: 0.0498
Epoch 11/20
9/9 [==============

37/37 [==============================] - 0s 967us/step - loss: 0.6931 - mse: 6.9547e-09
Epoch 16/20
37/37 [==============================] - 0s 1ms/step - loss: 0.6931 - mse: 3.2781e-09
Epoch 17/20
37/37 [==============================] - 0s 1ms/step - loss: 0.6931 - mse: 2.4632e-09
Epoch 18/20
37/37 [==============================] - 0s 1ms/step - loss: 0.6931 - mse: 1.9447e-09
Epoch 19/20
37/37 [==============================] - 0s 1ms/step - loss: 0.6931 - mse: 1.6641e-09
Epoch 20/20
3/3 [==============================] - 0s 1ms/step - loss: 0.6931 - mse: 0.0027
Loss: 0.6931474208831787
Mean Squared Error: 0.0027473827358335257
len of pred:  (91, 1) y_test:  (91,)
Mean Squared Error: 0.002747382618270931
Mean Absolute Error: 0.005580147544106284
R^2 Score: -0.011158909417848362
************************
 vancomycin
Epoch 1/20
67/67 [==============================] - 0s 850us/step - loss: 0.3527 - mse: 0.1025
Epoch 2/20
67/67 [==============================] - 0s 893us/step - loss: 0.

In [287]:
antibiotic_precent

{'chloramphenicol': 0.8508268383147382,
 'ciprofloxacin': 0.8901616339665916,
 'ceftiofur': 0.9201845849123188,
 'amoxicillin-clavulanic acid': 0.8420372349370869,
 'linezolid': 0.5219898616209999,
 'moxifloxacin': 0.697561348655386,
 'ceftazidime-clavulanic acid': 0.8592278110190537,
 'cefpodoxime': 0.8956508714995979,
 'nitrofurantoin': 0.7809815795597038,
 'clindamycin': 0.6034749385937574,
 'azithromycin': 0.7648399525166955,
 'cefotetan': 0.6374714974407787,
 'ampicillin-sulbactam': 0.5691193948696184,
 'cefotaxime-clavulanic acid': 0.816862937680481,
 'cefazolin': 0.7417300430579861,
 'ceftazidime': 0.7393697845528249,
 'tobramycin': 0.7911055869681399,
 'sulfamethoxazole': 0.8871142058162066,
 'teicoplanin': -0.6618370964132141,
 'rifampin': -0.019198334012066498,
 'tetracycline': 0.9176841347776014,
 'cefoxitin': 0.8436106162921411,
 'sulfisoxazole': 0.947007800584611,
 'amikacin': 0.648132360110915,
 'trimethoprim': 0.8977076536337382,
 'colistin': 0.4258046222407105,
 'doripe

In [294]:
gene_antibiotic_df[gene_antibiotic_df['chloramphenicol'] >0.9]


,chloramphenicol,dicloxacillin,ciprofloxacin,ceftiofur,fosfomycin-glucose-6-phosphate,amoxicillin-clavulanic acid,benzylpenicillin,metronidazole,linezolid,piperacillin,...,trimethoprim-sulfamethoxazole,aztreonam,norfloxacin,neomycin,Imipenem-EDTA-PA,delafloxacin,zoliflodacin,vancomycin,ertapenem,cefiderocol
floR,0.989191,-1,0.383982,0.029786,-1,0.003420,-1,-1,0.214189,-1,...,0.187397,0.211240,0.242684,-1,0.500366,-1.000000,-1.0,0.026373,0.181844,-1
blaKPC-3,0.915442,-1,0.000643,-1.000000,-1,0.999996,-1,-1,-1.000000,-1,...,0.035525,0.999147,0.497124,-1,0.500320,-1.000000,-1.0,-1.000000,0.999213,-1
acrB_R717L,0.981041,-1,0.000885,0.052554,-1,0.000296,-1,-1,-1.000000,-1,...,0.000421,-1.000000,-1.000000,-1,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1
ramR_T50P,0.964853,-1,0.008513,0.006366,-1,0.061850,-1,-1,-1.000000,-1,...,0.005394,-1.000000,-1.000000,-1,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1
blaACT-46,0.910554,-1,0.034263,-1.000000,-1,0.999488,-1,-1,-1.000000,-1,...,0.449539,0.905360,-1.000000,-1,-1.000000,-1.000000,-1.0,-1.000000,0.996365,-1
catA,0.969188,-1,0.068733,-1.000000,-1,0.953578,-1,-1,0.001171,-1,...,0.058672,0.358767,0.234233,-1,0.500712,0.588127,-1.0,0.001455,0.146779,-1
catA1,0.987824,-1,0.025530,0.038221,-1,0.090949,-1,-1,-1.000000,-1,...,0.084136,0.074413,0.532254,-1,0.501128,0.565999,-1.0,-1.000000,0.002486,-1
qnrA1,0.998487,-1,0.542146,0.068088,-1,0.000448,-1,-1,-1.000000,-1,...,0.198038,0.032708,-1.000000,-1,0.500406,-1.000000,-1.0,-1.000000,0.083709,-1
blaTEM-103,0.978650,-1,0.000827,0.015264,-1,0.000109,-1,-1,-1.000000,-1,...,0.458142,0.052150,-1.000000,-1,-1.000000,-1.000000,-1.0,-1.000000,0.066118,-1
blaNDM-7,0.953085,-1,0.815491,-1.000000,-1,0.940765,-1,-1,0.017239,-1,...,0.000313,0.036701,-1.000000,-1,0.501057,-1.000000,-1.0,0.051955,0.996282,-1


In [295]:

gene_antibiotic_df.to_csv("gene_antibiotic_df_precent.csv",index=False)

### model - accuraccy & MSE

In [32]:
predictions

array([[0.00322947],
       [0.9983469 ],
       [0.00322947],
       ...,
       [0.00265715],
       [0.00217324],
       [0.0024083 ]], dtype=float32)

### Evaluation - continues (MSE)

Mean Squared Error: 0.01619927366086918
Mean Absolute Error: 0.03154599644390347
R^2 Score: 0.8993035344650906


### one examle test

In [17]:
y_test[1]

1.0

In [18]:
model.predict(X_test[1].reshape(1, -1))

array([[0.99743843]], dtype=float32)

### Test for each Gene seperate

,qnrB48,blaADC-155,oqxB19,blaACT-37,blaTEM-19,cmx,rmtF2,blaCTX-M-24,qnrB77,blaOXY-2-6,...,gyrA_D95N,mgrB_M27K,blaCDD,blaOXA-735,pmrB_T140P,blaOXA-494,qnrB38,tmexD3,tet(O),murA_D278E
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1101,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
pred_gene = model.predict(df_one_gene)
data = pd.DataFrame(data=pred_gene)

In [50]:
data

,0
0,0.062887
1,0.002434
2,0.015267
3,0.001975
4,0.001146
...,...
1099,0.016005
1100,0.001224
1101,0.049610
1102,0.007509


In [69]:
genes_list = list(oxacillin_genes)

# Access the element at index 29
gene_at_index = genes_list[1]

print(gene_at_index)

blaADC-155


In [48]:
data = pd.DataFrame(data=pred_gene)
data[data > 0.5].dropna()

,0
8,0.642578
12,0.824058
29,0.997132
42,0.636477
90,0.988201
95,0.888557
146,0.908533
150,0.604366
195,0.978971
227,0.639507


In [36]:
pred_gene.max()

0.9973835

In [37]:
pred_gene.min()

1.4436484e-05

In [ ]:
# going over all the bacteria that are resistant to "chloramphenicol":
for gene_lst in df[(df.drug == "ciprofloxacin") & (df.resistance == 0)]['AMR genotypes'].values:
# going over all the genes of each bacteria :
    for g in gene_lst.split():
        print(g)
        v = np.zeros(X.shape[1])
        v[feature_names.index(g)]=1
        print(classifier.predict_proba([v]))
    print("***************************")